In [1]:
import numpy as np
import matplotlib.pyplot as plt
from Potential import Potential
from Metode.GreenFunction import GreenFunction
from Metode.AnalyticalMethod import AnalyticalMethod_V2
from Metode.MatrixTransfer import TransferMatrixMethod_V2
N  = 100
a = 6
m = 1
h = 1
E = 8
V_max = 10

get = Potential(N, a,E, m, h)
x = get.x
potential_barrier = get.PotentialBarrier(get,V_max,0)
step_potential = get.StepPotential(get,V_max,0)
morse_feshbach_potential_well = get.MorseFeshbach(get,V_max,1,0,0.2)

In [9]:
t = h**2/(2*m)*np.ones(N+1)
V1 = potential_barrier.get_potential()
from scipy.sparse import diags
T = diags([-t[:-1], V1+2*t, -t[:-1]], [-1, 0, 1], shape=(N,N)).toarray()
T

array([[ 1. , -0.5,  0. , ...,  0. ,  0. ,  0. ],
       [-0.5,  1. , -0.5, ...,  0. ,  0. ,  0. ],
       [ 0. , -0.5,  1. , ...,  0. ,  0. ,  0. ],
       ...,
       [ 0. ,  0. ,  0. , ...,  1. , -0.5,  0. ],
       [ 0. ,  0. ,  0. , ..., -0.5,  1. , -0.5],
       [ 0. ,  0. ,  0. , ...,  0. , -0.5,  1. ]])

In [10]:
from scipy.sparse import diags
from scipy.sparse.linalg import inv as inverse

class GreenFunction_2:
    def __init__(self, m, h, E):
        __name__ = "Green's Function Method"
        self.m = m
        self.h = h
        self.E = E
        self.t = self.h**2/(2*self.m)
    
    def Hamiltonian(self, V):
        N= V.parent.N
        V = V.get_potential()
        t = self.t*np.ones(N+1)
        H = diags([-t[:-1], V+2*t, -t[:-1]], [-1, 0, 1], shape=(N,N))
        return H
    
    def GreenFunction(self, V):
        N = V.parent.N
        E = self.E
        H = self.Hamiltonian(V)
        G = inverse(E*np.eye(N) - H)
        return G

In [12]:
Green2 = GreenFunction_2(m=1,h=1,E=E)
H = Green2.Hamiltonian(potential_barrier).toarray()
G = Green2.GreenFunction(potential_barrier)

TypeError: Input must be a sparse matrix